In [7]:
#import os
#os.chdir('../')

In [1]:
# Modify the sys.path
import sys
sys.path.append(r"C:\Users\Alice\Desktop\Code\eptic\src")

# Import the necessary functions
from db_connect import get_database_config_from_json, connect_to_db

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

import hydra
from omegaconf import DictConfig

import mariadb
import json

Notebook to figure out what's wrong with the "duration" field of some languages

In [ ]:
from hydra import initialize, compose
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="conf/"):
    cfg = compose(config_name='config.yaml')
    print(OmegaConf.to_yaml(cfg))

In [24]:
hydra.core.global_hydra.GlobalHydra.instance().clear()

In [2]:
from hydra import initialize, compose

# Use the 'initialize' and 'compose' functions for Hydra
config_dir_path = r"C:\Users\Alice\Desktop\Code\eptic\config"

with initialize(version_base=None, config_path="../config"):
    cfg = compose(config_name='main.yaml')  # Adjust this if your filename differs

config_data = get_database_config_from_json(cfg.db.credentials)
print(f"Connecting to database: {config_data['name']}")
conn = connect_to_db(config_data)

sql_query = "SELECT lang FROM texts"
df = pd.read_sql_query(sql_query, conn)

output_path = "temp_dataframe.csv"
df.to_csv(output_path, index=False)

df  # Display the dataframe

Connecting to database: eptic
<class 'sqlalchemy.engine.base.Connection'>


,lang
0,pl
1,pl
2,fr
3,fr
4,en
...,...
976,fi
977,fi
978,fi
979,fi
